In [1]:
import numpy as np
import pandas as pd
from pyquery import PyQuery as pq
import time
import datetime
import requests
import urllib.request
import os
import calendar
import pickle

In [2]:
# input
ETF_data = pd.read_csv('Emerging Asia Pacific ETF List (114).csv')

# filter the inception date
ETF_data['Inception'] = pd.to_datetime(ETF_data['Inception'], format="%d/%m/%Y")
date = datetime.datetime(2016, 1, 1, 0, 0)
ETF_data = ETF_data.loc[ETF_data['Inception'] < date,:].reset_index()

# extract symbol and name of each ETF
symbol_list = ETF_data['Symbol']
name_list = ETF_data['ETF Name']

In [3]:
with open('ETF_dic.pkl','rb') as f:
    dic = pickle.load(f)

In [19]:
browser=webdriver.Chrome('/Users/lou/Downloads/chromedriver') 
browser.get('https://finance.yahoo.com/')
browser.find_element_by_name('yfin-usr-qry').send_keys(dic['name'][0]+'\n')
# link = browser.
browser.find_element_by_class_name('')
browser.quit()

In [139]:
def get_the_last_day_of_month(start_date,end_date):
    ans = []
    temp = start_date - datetime.timedelta(1)
    year = start_date.year
    month = start_date.month
    
    while (temp < end_date):
        temp = temp + datetime.timedelta(calendar.monthrange(year,month)[1])
        ans.append(temp)
        month += 1
        if(month > 12):
            month = 1
            year += 1
    
    # check for trading day
    for i in range(len(ans)):
        if ans[i].isoweekday()>5:
            ans[i]=ans[i]+datetime.timedelta(days=5-ans[i].isoweekday())
    return ans

In [140]:
datetime.datetime(2018,12,31,0,0).isoweekday()

1

In [141]:
last_day_of_month = get_the_last_day_of_month(datetime.datetime(2016,1,1,0,0),datetime.datetime(2018,12,31,0,0))

In [234]:
def get_single_symbol(symbol,chrome_filepath,browser,wait = 3):
    
    # check directory
    if not os.path.isdir('./data'):
        os.mkdir('./data')
    
    filepath = f'./data/{symbol}.csv'
    if os.path.isfile(filepath):
        print('File exist')
        return 
    
    url = f'https://finance.yahoo.com/quote/{symbol}/history?p={symbol}'
    browser.get(url)
    browser.find_element_by_css_selector("[class='C(t) O(n):f Tsh($actionBlueTextShadow) Bd(n) Bgc(t) Fz(14px) Pos(r) T(-1px) Bd(n):f Bxsh(n):f Cur(p) W(190px)']").click()
    start_date = browser.find_element_by_css_selector("[class='Bdrs(0) Bxsh(n)! Fz(s) Bxz(bb) D(ib) Bg(n) Pend(5px) Px(8px) Py(0) H(34px) Lh(34px) Bd O(n):f O(n):h Bdc($c-fuji-grey-c) Bdc($c-fuji-blue-1-b):f M(0) Pstart(10px) Bgc(white) W(90px) Mt(5px)']")
    start_date.clear()
    start_date.send_keys('1/1/2016')
    end_date = browser.find_element_by_css_selector("[class='Bdrs(0) Bxsh(n)! Fz(s) Bxz(bb) D(ib) Bg(n) Pend(5px) Px(8px) Py(0) H(34px) Lh(34px) Bd O(n):f O(n):h Bdc($c-fuji-grey-c) Bdc($c-fuji-blue-1-b):f M(0) Pstart(10px) Bgc(white) W(90px) Mt(5px) Mstart(15px)']")
    end_date.clear()
    end_date.send_keys('1/1/2019')

    try:
        browser.find_element_by_css_selector("[class=' Bgc($c-fuji-blue-1-b) Bdrs(3px) Px(20px) Miw(100px) Whs(nw) Fz(s) Fw(500) C(white) Bgc($actionBlueHover):h Bd(0) D(ib) Cur(p) Td(n)  Py(9px) Miw(80px)! Fl(start)']").click()
    except:
        browser.find_element_by_css_selector("[class=' Bd Bdc($c-fuji-blue-1-b) Bdrs(3px) Px(20px) Miw(100px) Whs(nw) Fz(s) Fw(500) D(ib) C($c-fuji-blue-1-b) Bdc($actionBlueHover):h C($actionBlueHover):h Cur(p) Td(n)  Py(8px) cancel Miw(80px)! Fl(end)']").click()
        browser.find_element_by_css_selector("[class='C(t) O(n):f Tsh($actionBlueTextShadow) Bd(n) Bgc(t) Fz(14px) Pos(r) T(-1px) Bd(n):f Bxsh(n):f Cur(p) W(190px)']").click()
        browser.find_element_by_css_selector("[class=' Bgc($c-fuji-blue-1-b) Bdrs(3px) Px(20px) Miw(100px) Whs(nw) Fz(s) Fw(500) C(white) Bgc($actionBlueHover):h Bd(0) D(ib) Cur(p) Td(n)  Py(9px) Miw(80px)! Fl(start)']").click()

    browser.find_element_by_css_selector("[class=' Bgc($c-fuji-blue-1-b) Bdrs(3px) Px(20px) Miw(100px) Whs(nw) Fz(s) Fw(500) C(white) Bgc($actionBlueHover):h Bd(0) D(ib) Cur(p) Td(n)  Py(9px) Fl(end)']").click()
    time.sleep(5)
    browser.find_element_by_css_selector("[class='Fl(end) Mt(3px) Cur(p)']").click()
    time.sleep(wait)

    while not os.path.isfile(filepath):
        browser.find_element_by_css_selector("[class='Fl(end) Mt(3px) Cur(p)']").click()
        time.sleep(wait)

    print('Download',symbol)

In [225]:
def get_all_symbol_csv(dic,chrome_filepath):
    
    # check directory
    if not os.path.isdir('./data'):
        os.mkdir('./data')

    options = webdriver.ChromeOptions()
    prefs = {'download.default_directory': './data'}
    options.add_experimental_option('prefs', prefs)
    browser=webdriver.Chrome(chrome_filepath,options=options) 
    
    for s in dic['symbol']:
        get_single_symbol(s,check_download_file,browser)

    browser.quit()

In [222]:
get_all_symbol_csv(dic,'/Users/lou/Downloads/chromedriver')

File exist
Download INCO
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist


In [226]:
def combine_data(dic,last_day_of_month):
    com_data = pd.DataFrame(last_day_of_month[::-1],columns=['Date'])
        
    for s in dic['symbol']:
        data = pd.read_csv(f'./data/{s}.csv')
        data['Date'] = pd.to_datetime(data['Date'], format="%Y-%m-%d")
        
        # Mask
        mask = [False for i in range(data['Date'].shape[0])]
        for day in last_day_of_month:
            temp = data['Date'] == day
            mask |= temp
        data = data[mask]
        
        data_close = data[['Date','Close']].reset_index(drop=True)
        data_close = data_close.rename(columns={'Close':s})
        com_data = com_data.merge(data_close,on='Date',how='left')
    return com_data

In [235]:
def check_download_file(com_data,chrome_filepath):
    options = webdriver.ChromeOptions()
    prefs = {'download.default_directory': './data'}
    options.add_experimental_option('prefs', prefs)
    browser=webdriver.Chrome(chrome_filepath,options=options) 
    
    for s in dic['symbol']:
        data = com_data[s].values
        if (np.sum(np.isnan(data)) > 10):
            os.remove(f'./data/{s}.csv')
            get_single_symbol(s,chrome_filepath,browser,5)
    browser.quit()

In [236]:
check_download_file(com_data,'/Users/lou/Downloads/chromedriver')

Download NFTY
Download CHIC
Download KALL


In [232]:
com_data = combine_data(dic,last_day_of_month)

In [233]:
com_data

,Date,XCEM,INCO,INXX,SCIN,AFTY,CNHX,YANG,YINN,CHAD,...,CXSE,DGRE,DEM,DGS,EPI,ASHX,ASHR,ASHS,CN,DBEM
0,2018-12-31,23.770000,44.200001,11.91,14.920000,13.110000,24.080000,66.669998,17.389999,39.779999,...,60.889999,22.219999,40.150002,41.889999,24.799999,15.879000,21.930000,22.330000,28.384001,20.889999
1,2018-11-30,24.799999,43.650002,11.92,14.600000,14.190000,23.920000,56.099998,21.260000,37.988998,...,66.639999,23.091999,41.369999,43.279999,24.430000,16.570000,23.420000,23.200001,30.695000,22.010000
2,2018-10-31,24.340000,38.880001,11.02,14.260000,14.305000,23.530001,71.300003,17.790001,38.570000,...,63.230000,21.785000,40.340000,41.040001,22.459999,16.580000,23.230000,22.600000,29.430000,21.309999
3,2018-09-28,25.895000,41.000000,11.87,14.150000,15.354000,24.650000,57.009998,23.840000,35.799999,...,71.669998,23.940001,43.669998,45.650002,24.110001,18.100000,25.350000,25.450001,32.556000,23.080000
4,2018-08-31,25.639999,47.900002,13.35,17.840000,14.700000,24.650000,59.650002,23.450001,37.060001,...,74.370003,24.790001,43.759998,47.209999,26.830000,17.500000,24.740000,25.629999,32.612000,23.490000
5,2018-07-31,26.160000,47.880001,13.10,18.200001,15.329000,26.170000,56.090000,26.240000,34.930000,...,78.370003,25.566000,45.290001,48.790001,26.780001,18.610001,26.500000,28.059999,34.630001,23.629999
6,2018-06-29,24.969999,45.840000,12.61,16.920000,15.260000,25.920000,60.220001,25.400000,35.270000,...,81.660004,24.850000,42.869999,47.689999,25.170000,18.940001,26.530001,28.559999,35.025002,23.160000
7,2018-05-31,25.938000,46.590000,13.50,19.070000,16.844999,27.839001,49.580002,31.900000,31.770000,...,87.739998,26.040001,45.439999,51.669998,25.719999,20.870001,29.600000,32.919998,37.960999,23.820000
8,2018-04-30,27.459999,48.750000,14.36,20.510000,16.930000,27.299999,49.290001,32.820000,31.719999,...,84.199997,26.459999,46.320000,52.240002,26.760000,20.600000,29.799999,33.880001,37.334999,24.129999
9,2018-03-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [185]:
data = combine_data(dic,last_day_of_month)[dic['symbol'][1]].values

In [190]:
np.sum(np.isnan(data))

36